In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import agate

In [3]:
%%bash
## download the mixedbev file
curl -L -O https://comptroller.texas.gov/auto-data/odc/MIXEDBEV_02_2017.CSV

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2565k  100 2565k    0     0  1379k      0  0:00:01  0:00:01 --:--:-- 1414k
/Users/christian/anaconda/envs/mixbev/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-03-26 20:00:59.998856. Please add timezone info to timestamps.


In [6]:
%%bash
head -n 5 MIXEDBEV_02_2017.CSV

"MB821424    ","ABI-HAUS                      ","959 N 2ND ST                  ","ABILENE             ","TX","79601","221","          ","2016/12", 000000637.97
"MB638028    ","ABILENE BEEHIVE INC           ","442 CEDAR ST STE A            ","ABILENE             ","TX","79601","221","          ","2017/01", 000002557.12
"MB543114    ","ABILENE BOWLING LANES INC     ","279 RUIDOSA AVE               ","ABILENE             ","TX","79605","221","          ","2017/01", 000000287.49
"MB933130    ","ABILENE CABARET LLC           ","1918 BUTTERNUT ST             ","ABILENE             ","TX","79602","221","          ","2017/01", 000000988.04
"N 037863    ","ABILENE COUNTRY CLUB          ","4039 S TREADAWAY BLVD         ","ABILENE             ","TX","79602","221","          ","2017/01", 000002068.82


### Study variables
These are set here depending on what city or county you want to study

In [111]:
# this is our source file, which may have been downloaded above
file = 'MIXEDBEV_02_2017.CSV'

# city studied needs to be all caps
city_studied = 'ROUND ROCK'

# county studied
county_studied = 'BASTROP'

### Data variables
These probalby won't change between analysis or data sets

In [94]:
column_names = [
    'TABC Permit Number',
    'Trade Name',
    'Location Address',
    'Location City',
    'Location State',
    'Location Zip Code',
    'Location County Code',
    'Blank',
    'Report Period',
    'Report Tax'
]
specified_types = {
    'Location Zip Code': agate.Text(),
    'Location County Code': agate.Text()
}

### Processing the file

In [98]:
# this imports the file specified above
mixbev_raw = agate.Table.from_csv(file, column_names, encoding='iso-8859-1', column_types=specified_types)
# prints table fields
print(mixbev_raw)

| column               | data_type |
| -------------------- | --------- |
| TABC Permit Number   | Text      |
| Trade Name           | Text      |
| Location Address     | Text      |
| Location City        | Text      |
| Location State       | Text      |
| Location Zip Code    | Text      |
| Location County Code | Text      |
| Blank                | Boolean   |
| Report Period        | Text      |
| Report Tax           | Number    |



In [99]:
# This creates a new interim table with results of compute function
# that takes the four columns that need trimming and strips them
# adding them to the end of the table with new names
mixbev_trim = mixbev_raw.compute([
    ('Permit', agate.Formula(agate.Text(), lambda r: r['TABC Permit Number'].strip())),
    ('Name', agate.Formula(agate.Text(), lambda r: r['Trade Name'].strip())),
    ('Address', agate.Formula(agate.Text(), lambda r: r['Location Address'].strip())),
    ('City', agate.Formula(agate.Text(), lambda r: r['Location City'].strip()))
])

In [11]:
## shows the new columns added to the interim table
print(mixbev_trim)

| column               | data_type |
| -------------------- | --------- |
| TABC Permit Number   | Text      |
| Trade Name           | Text      |
| Location Address     | Text      |
| Location City        | Text      |
| Location State       | Text      |
| Location Zip Code    | Text      |
| Location County Code | Text      |
| Blank                | Boolean   |
| Report Period        | Text      |
| Report Tax           | Number    |
| Permit               | Text      |
| Name                 | Text      |
| Address              | Text      |
| City                 | Text      |



In [12]:
## creates new table with just stuff we need with clean names
# new_table = table.select(['3rd_column_name', '1st_column_name', '2nd_column_name'])
mixbev_cleaned = mixbev_trim.select([
    'Permit',
    'Name',
    'Address',
    'City',
    'Location State',
    'Location County Code',
    'Report Period',
    'Report Tax'
]).rename(column_names = {
    'Location State': 'State',
    'Location County Code': 'CountyCode',
    'Report Period': 'Period',
    'Report Tax': 'Tax'
})

In [13]:
## these are now the columns present in our new column
print(mixbev_cleaned)

| column     | data_type |
| ---------- | --------- |
| Permit     | Text      |
| Name       | Text      |
| Address    | Text      |
| City       | Text      |
| State      | Text      |
| CountyCode | Text      |
| Period     | Text      |
| Tax        | Number    |



In [14]:
# and this peeks at the data
# I did send this to_csv and made sure columns were trimmed
mixbev_cleaned.limit(5).print_table()

| Permit   | Name                 | Address              | City    | State | CountyCode | ... |
| -------- | -------------------- | -------------------- | ------- | ----- | ---------- | --- |
| MB638028 | ABILENE BEEHIVE INC  | 442 CEDAR ST STE A   | ABILENE | TX    | 221        | ... |
| MB543114 | ABILENE BOWLING L... | 279 RUIDOSA AVE      | ABILENE | TX    | 221        | ... |
| MB933130 | ABILENE CABARET LLC  | 1918 BUTTERNUT ST    | ABILENE | TX    | 221        | ... |
| N 037863 | ABILENE COUNTRY CLUB | 4039 S TREADAWAY ... | ABILENE | TX    | 221        | ... |
| MB200506 | ABILENE SEAFOOD T... | 1882 S CLACK ST      | ABILENE | TX    | 221        | ... |


### Create establishment column

We do this so we make sure we have single establishments instead of grouping trade names together from different addresses, like 'CHILI'S BAR & GRILL'.

In [74]:
mixbev_cleaned_est = mixbev_cleaned.compute([
    ('Establishment', agate.Formula(agate.Text(), lambda r: '%(Name)s %(Address)s' % r))
])

In [75]:
print(mixbev_cleaned_est)

| column        | data_type |
| ------------- | --------- |
| Permit        | Text      |
| Name          | Text      |
| Address       | Text      |
| City          | Text      |
| State         | Text      |
| CountyCode    | Text      |
| Period        | Text      |
| Tax           | Number    |
| Establishment | Text      |



In [76]:
mixbev_establishment = mixbev_cleaned_est.select('Establishment')
mixbev_establishment.print_table(max_column_width=80)

| Establishment                                      |
| -------------------------------------------------- |
| ABILENE BEEHIVE INC 442 CEDAR ST STE A             |
| ABILENE BOWLING LANES INC 279 RUIDOSA AVE          |
| ABILENE CABARET LLC 1918 BUTTERNUT ST              |
| ABILENE COUNTRY CLUB 4039 S TREADAWAY BLVD         |
| ABILENE SEAFOOD TAVERN 1882 S CLACK ST             |
| ABUELO'S BEVERAGE CORPORATION 4782 S 14TH ST       |
| ACE IN THE HOLE 133 EPLENS CT                      |
| BILLIARDS PLUS 5495 S 7TH ST                       |
| BONZAI JAPANESE STEAK HOUSE 1802 S CLACK ST        |
| BREAKERS SPORTS BAR 1874 S CLACK ST                |
| BUFFALO WILD WINGS GRILL & BAR 1010 E OVERLAND TRL |
| BUFFALO WILD WINGS GRILL AND B 4401 RIDGEMONT DR   |
| CAHOOTS CATFISH & OYSTER BAR/J 301 S 11TH ST       |
| CASA HERRERA 4109 RIDGEMONT DR                     |
| CHELSEA'S ST PUB 4310 BUFFALO GAP RD STE 1342      |
| CHELSEA'S ST PUB 4310 BUFFALO GAP RD STE 1342      |
| CHILI'S 

In [77]:
# importing countes.csv, ensuring that the 'code' column is text
counties = agate.Table.from_csv('counties.csv', column_types={'code': agate.Text()}).rename()

In [78]:
print(counties)

| column | data_type |
| ------ | --------- |
| id     | Number    |
| county | Text      |
| code   | Text      |



In [79]:
counties.print_table()

| id | county    | code |
| -- | --------- | ---- |
|  1 | Anderson  | 001  |
|  2 | Andrews   | 002  |
|  3 | Angelina  | 003  |
|  4 | Aransas   | 004  |
|  5 | Archer    | 005  |
|  6 | Armstrong | 006  |
|  7 | Atascosa  | 007  |
|  8 | Austin    | 008  |
|  9 | Bailey    | 009  |
| 10 | Bandera   | 010  |
| 11 | Bastrop   | 011  |
| 12 | Baylor    | 012  |
| 13 | Bee       | 013  |
| 14 | Bell      | 014  |
| 15 | Bexar     | 015  |
| 16 | Blanco    | 016  |
| 17 | Borden    | 017  |
| 18 | Bosque    | 018  |
| 19 | Bowie     | 019  |
| 20 | Brazoria  | 020  |
| ... | ...       | ...  |


In [80]:
mixbev_joined = mixbev_cleaned_est.join(counties, 'CountyCode', 'code')

In [81]:
print(mixbev_joined)

| column        | data_type |
| ------------- | --------- |
| Permit        | Text      |
| Name          | Text      |
| Address       | Text      |
| City          | Text      |
| State         | Text      |
| CountyCode    | Text      |
| Period        | Text      |
| Tax           | Number    |
| Establishment | Text      |
| id            | Number    |
| county        | Text      |



In [82]:
mixbev = mixbev_joined.select([
    'Permit',
    'Name',
    'Address',
    'Establishment',
    'City',
    'State',
    'county',
    'Period',
    'Tax'
]).rename(column_names = {
    'county': 'County'
})

In [83]:
print(mixbev)

| column        | data_type |
| ------------- | --------- |
| Permit        | Text      |
| Name          | Text      |
| Address       | Text      |
| Establishment | Text      |
| City          | Text      |
| State         | Text      |
| County        | Text      |
| Period        | Text      |
| Tax           | Number    |



In [84]:
mixbev.print_table()

| Permit   | Name                 | Address              | Establishment        | City    | State | ... |
| -------- | -------------------- | -------------------- | -------------------- | ------- | ----- | --- |
| MB638028 | ABILENE BEEHIVE INC  | 442 CEDAR ST STE A   | ABILENE BEEHIVE I... | ABILENE | TX    | ... |
| MB543114 | ABILENE BOWLING L... | 279 RUIDOSA AVE      | ABILENE BOWLING L... | ABILENE | TX    | ... |
| MB933130 | ABILENE CABARET LLC  | 1918 BUTTERNUT ST    | ABILENE CABARET L... | ABILENE | TX    | ... |
| N 037863 | ABILENE COUNTRY CLUB | 4039 S TREADAWAY ... | ABILENE COUNTRY C... | ABILENE | TX    | ... |
| MB200506 | ABILENE SEAFOOD T... | 1882 S CLACK ST      | ABILENE SEAFOOD T... | ABILENE | TX    | ... |
| MB541702 | ABUELO'S BEVERAGE... | 4782 S 14TH ST       | ABUELO'S BEVERAGE... | ABILENE | TX    | ... |
| MB932373 | ACE IN THE HOLE      | 133 EPLENS CT        | ACE IN THE HOLE 1... | ABILENE | TX    | ... |
| MB248134 | BILLIARDS PLUS       | 5495 S 7TH

### Looking at dates of the records

Here we have to:
- create a tableset using group_by by the period
- creaet a table using aggregate function to count
- create a table to sort the period in reverse order
- Then print the sorted table (top 10 rows)

In [86]:
by_period = mixbev.group_by('Period')

period_totals = by_period.aggregate([
    ('count', agate.Count())
])

period_totals_sorted = period_totals.order_by('count', reverse=True)

period_totals_sorted.limit(10).print_table(max_rows=None)


| Period  |  count |
| ------- | ------ |
| 2017/01 | 14,317 |
| 2016/12 |  1,573 |
| 2016/11 |    198 |
| 2016/10 |     71 |
| 2017/02 |     33 |
| 2016/09 |     28 |
| 2016/08 |     17 |
| 2016/07 |     10 |
| 2016/03 |      9 |
| 2016/04 |      7 |


We have an answer here that we need for the future, and that is the period of time that has the most records. If we are studying a particular month of records, we need to set that so we can change it later with a different dataset.

In [39]:
# setting the month_studied var:
month_studied = '2017/01'

Now we can filter or select the records where the month equals what we want.

In [87]:
mixbev_month = mixbev.where(lambda row: row['Period'] == month_studied)
len(mixbev_month)

14317

### Tops sales in a city

Uses the `city_studied` variable at the top of the workbook

In [115]:
# Filters the data to the specified city
mixbev_city = mixbev_month.where(lambda row: row['City'] == city_studied)

# groups the data based on Establishment and City
city_grouped = mixbev_city.group_by('Establishment').group_by('City')
# computes the sales based on the grouping
summary = city_grouped.aggregate([
    ('total_sales', agate.Sum('Tax'))
])
# sorts the results by most sold
summary_sorted = summary.order_by('total_sales', reverse=True)
# prints the top 10 results
summary_sorted.limit(10).print_table(max_column_width=80)


| Establishment                                           | City       | total_sales |
| ------------------------------------------------------- | ---------- | ----------- |
| THIRD BASE ROUND ROCK, LLC 3107 S INTERSTATE 35 STE 810 | ROUND ROCK |    9,368.81 |
| JACK ALLEN'S KITCHEN 2500 HOPPE TRL                     | ROUND ROCK |    8,730.10 |
| RICK'S CABARET 3105 S INTERSTATE 35                     | ROUND ROCK |    8,359.45 |
| CHUY'S ROUND ROCK 2320 N INTERSTATE 35                  | ROUND ROCK |    8,273.96 |
| SALT TRADERS 2850 N INTERSTATE 35                       | ROUND ROCK |    8,109.21 |
| TWIN PEAKS RESTAURANT 100 LOUIS HENNA BLVD              | ROUND ROCK |    7,917.25 |
| FAST EDDIE'S NEIGHBORHOOD BILL 100 PARKER DR            | ROUND ROCK |    7,671.96 |
| PLUCKERS WING BAR 300 MAYS XING STE 300                 | ROUND ROCK |    7,358.00 |
| COVER 3 2800 N INTERSTATE 35 STE 200                    | ROUND ROCK |    6,212.37 |
| THE BRASS TAP 204 E MAIN ST              